# 构建具有上下文感知能力的对话代理

## 概述
本教程概述了创建一个能够在多次交互中保持上下文的对话代理的过程。我们将使用现代 AI 框架来构建一个能够进行更自然、连贯对话的代理。

## 动机
许多简单的聊天机器人缺乏保持上下文的能力，导致用户体验脱节且令人沮丧。本教程旨在通过实现一个能够记住并引用对话先前部分的对话代理来解决这个问题，从而提高整体交互质量。

## 关键组件
1. **语言模型**：生成响应的核心 AI 组件。
2. **提示模板**：定义我们对话的结构。
3. **历史管理器**：管理对话历史和上下文。
4. **消息存储**：存储每个对话会话的消息。

## 方法详情

### 设置环境
首先设置必要的 AI 框架并确保能够访问合适的语言模型。这构成了我们对话代理的基础。

### 创建聊天历史存储
实现一个系统来管理多个对话会话。每个会话都应具有唯一标识，并与其自己的消息历史相关联。

### 定义对话结构
创建一个包含以下内容的模板：
- 定义 AI 角色的系统消息
- 对话历史的占位符
- 用户的输入

这种结构指导 AI 的响应并在整个对话过程中保持一致性。

### 构建对话链
将提示模板与语言模型结合以创建一个基本的对话链。将此链包装在历史管理组件中，该组件自动处理对话历史的插入和检索。

### 与代理交互
要使用该代理，请使用用户输入和会话标识符调用它。历史管理器负责检索适当的对话历史，将其插入提示中，并在每次交互后存储新消息。

## 结论
这种创建对话代理的方法提供了几个优点：
- **上下文感知**：代理可以引用对话的先前部分，从而实现更自然的交互。
- **简单性**：模块化设计使实现保持简单。
- **灵活性**：易于修改对话结构或切换到不同的语言模型。
- **可扩展性**：基于会话的方法允许管理多个独立的对话。

有了这个基础，您可以通过以下方式进一步增强代理：
- 实现更复杂的提示工程
- 将其与外部知识库集成
- 为特定领域添加专业能力
- 结合错误处理和对话修复策略

通过专注于上下文管理，这种对话代理设计显着改进了基本的聊天机器人功能，为更具吸引力和有用的 AI 助手铺平了道路。

# 对话代理教程

本笔记本演示了如何使用 LangChain 创建一个简单的对话代理。

### 导入所需的库

In [10]:
# %pip install -q langchain langchain_experimental openai python-dotenv langchain_openai

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from dotenv import load_dotenv
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### 加载环境变量并初始化语言模型

In [11]:
load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000, temperature=0)

###  创建一个简单的内存聊天历史存储


In [12]:
store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

### 创建提示模板


In [13]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

### 将提示和模型组合成可运行的链


In [14]:
chain = prompt | llm

### 用消息历史包装链


In [15]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

### 示例用法

In [16]:
session_id = "user_123"


response1 = chain_with_history.invoke(
    {"input": "Hello! How are you?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response1.content)

response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2.content)


AI: Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
AI: Your previous message was, "Hello! How are you?" How can I assist you further?


### 打印对话历史

In [17]:
print("\nConversation History:")
for message in store[session_id].messages:
    print(f"{message.type}: {message.content}")


Conversation History:
human: Hello! How are you?
ai: Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
human: What was my previous message?
ai: Your previous message was, "Hello! How are you?" How can I assist you further?
